In [1]:

# Libraries dependencies
import MetaTrader5 as mt5
from datetime import datetime, timezone
import pandas as pd
import numpy as np

# Project files
from AlgorithmicTrading.models.metatrader import (
    MqlTradeRequest, 
    MqlPositionInfo, 
    MqlAccountInfo, 
    MqlTradeOrder, 
    MqlTick, 
    MqlSymbolInfo,   
    ENUM_ORDER_TYPE_MARKET,
    ENUM_POSITION_TYPE, 
    ENUM_DEAL_ENTRY,
    ENUM_DEAL_TYPE, 
    ENUM_TIMEFRAME, 
    ENUM_ACCOUNT_MARGIN_MODE
)
from AlgorithmicTrading.trade.trade import Trade
from AlgorithmicTrading.account import AccountBacktest, AccountLive
from AlgorithmicTrading.rates import Rates
from AlgorithmicTrading.utils.trades import get_last_tick


In [2]:
# Hedge account
# AccountLive.login(login=5013424139, server="MetaQuotes-Demo", password="pndvym2b")
# Netting account
account = AccountLive.login(login=5013526569, server="MetaQuotes-Demo", password="h3xhhvuo")
# JPY account
# account = AccountLive.login(login=5013973322, server="MetaQuotes-Demo", password="ppo0zpnk")
# Backtest account
account = AccountBacktest.login(balance=100_000, margin_mode=ENUM_ACCOUNT_MARGIN_MODE.ACCOUNT_MARGIN_MODE_RETAIL_NETTING)
account.orders

[INFO]: Successfull login to #5013526569 account
[INFO]: Successfull backtest account create


[]

In [3]:
n_rates = Rates.get_last_n_candles("EURUSD")
n_rates

,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-04-17 03:45:00+00:00,1.09840,1.09843,1.09818,1.09819,110,1,0
1,2023-04-17 03:50:00+00:00,1.09819,1.09821,1.09782,1.09786,166,0,0
2,2023-04-17 03:55:00+00:00,1.09790,1.09791,1.09727,1.09727,265,0,0
3,2023-04-17 04:00:00+00:00,1.09731,1.09759,1.09703,1.09722,201,0,0
4,2023-04-17 04:05:00+00:00,1.09721,1.09724,1.09663,1.09663,172,0,0
...,...,...,...,...,...,...,...,...
9995,2023-06-02 23:30:00+00:00,1.07053,1.07058,1.07052,1.07055,36,1,0
9996,2023-06-02 23:35:00+00:00,1.07056,1.07058,1.07054,1.07056,51,1,0
9997,2023-06-02 23:40:00+00:00,1.07055,1.07066,1.07053,1.07059,64,0,0
9998,2023-06-02 23:45:00+00:00,1.07058,1.07067,1.07057,1.07058,73,1,0


In [4]:
trade_class = Trade(account, 101010, 5, backtest_financial_data = n_rates.iloc[:-1])


In [5]:
trade_class.buy_stop("EURUSD", 2, 2, 3, 3)

ValidationError: 1 validation error for MqlTradeOrder
__root__
  Invalid stop loss (type=value_error)

In [8]:
account.orders[1].dict()

{'ticket': 1685880553635173,
 'time_setup': datetime.datetime(2023, 6, 4, 12, 9, 13, tzinfo=datetime.timezone.utc),
 'time_setup_msc': datetime.datetime(2023, 6, 4, 12, 9, 13, 635173, tzinfo=datetime.timezone.utc),
 'time_done': None,
 'time_done_msc': None,
 'time_expiration': None,
 'type': <ENUM_ORDER_TYPE.ORDER_TYPE_BUY_STOP: 4>,
 'type_time': <ENUM_ORDER_TYPE_TIME.ORDER_TIME_GTC: 0>,
 'type_filling': <ENUM_ORDER_TYPE_FILLING.ORDER_FILLING_FOK: 0>,
 'state': <ENUM_ORDER_STATE.ORDER_STATE_PLACED: 1>,
 'magic': 101010,
 'position_id': None,
 'position_by_id': None,
 'reason': <ENUM_ORDER_REASON.ORDER_REASON_EXPERT: 3>,
 'volume_initial': 2.0,
 'volume_current': 2.0,
 'price_open': 2.0,
 'sl': 3.0,
 'tp': 3.0,
 'price_current': 2.0,
 'price_stoplimit': 0.0,
 'symbol': 'EURUSD',
 'comment': '',
 'external_id': None}

In [26]:
deals = pd.DataFrame([deal.dict() for deal in account.history_deals])
deals["type"] = deals.type.apply(lambda value: ENUM_DEAL_TYPE(value).name)
deals["entry"] = deals.entry.apply(lambda value: ENUM_DEAL_ENTRY(value).name)
features = deals[["ticket", "time", "entry", "type", "position_id", "volume", "price", "profit"]]
features

,ticket,time,entry,type,position_id,volume,price,profit
0,1685840358253963,2023-06-04 00:59:18+00:00,DEAL_ENTRY_IN,DEAL_TYPE_BUY,1685840358251332,10.0,1.07080,0.0
1,1685840374261610,2023-06-04 00:59:34+00:00,DEAL_ENTRY_OUT,DEAL_TYPE_SELL,1685840358251332,5.0,1.07052,-140.0
2,1685840386747515,2023-06-04 00:59:46+00:00,DEAL_ENTRY_INOUT,DEAL_TYPE_SELL,1685840358251332,10.0,1.07052,-140.0
3,1685840386753018,2023-06-04 00:59:46+00:00,DEAL_ENTRY_IN,DEAL_TYPE_SELL,1685840358251332,5.0,1.07052,0.0
4,1685840406587139,2023-06-04 01:00:06+00:00,DEAL_ENTRY_OUT,DEAL_TYPE_BUY,1685840358251332,5.0,1.07059,-35.0
5,1685840543147675,2023-06-04 01:02:23+00:00,DEAL_ENTRY_IN,DEAL_TYPE_BUY,1685840543145467,5.0,1.07059,0.0
